This John Hopkins University (JHU) Covid data extract is based on the tutorial by B Chen which can be found at the following link
*    https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6 
    
It is used with his permission subject to the copyright conditions at the following link:
*    link goes here
#    need to track down Mit / B Chen link for attribution

The JHU data extract uses the following csv files:
*    time_series_covid19_confirmed_global.csv................confirmed global Covid_19 cases
*    time_series_covid19_deaths_global.csv...................confirmed global Covid_19 deaths
*    time_series_covid19_recovered_global.csv................confirmed global Covid_19 recovered 


In [1]:
#   import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from scipy.stats import linregress
from urllib.error import HTTPError
import numpy as np
import wget
import time
from datetime import datetime
from pandas_profiling import ProfileReport

#   the project uses urls / wget downloads in place of API calls are they are not available for the datasets needed

# url of the raw csv dataset
urls = [
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
]
[wget.download(url) for url in urls]



['time_series_covid19_confirmed_global.csv',
 'time_series_covid19_deaths_global.csv',
 'time_series_covid19_recovered_global.csv']

In [2]:
#   create dataframes from csv's

confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
    
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
    
recovered_df = pd.read_csv('time_series_covid19_recovered_global.csv')

confirmed_df


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/25/22,1/26/22,1/27/22,1/28/22,1/29/22,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,160252,160692,161004,161057,161290,162111,162926,163555,164190,164727
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,248859,251015,252577,254126,254126,255741,258543,258543,261240,261240
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,241406,243568,245698,247568,249310,250774,252117,253520,254885,255836
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,34701,35028,35028,35556,35556,35556,35958,35958,36315,36470
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,97594,97812,97901,98029,98057,98076,98116,98226,98267,98319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,...,2171527,2187481,2203208,2218137,2233287,2263053,2275727,2286750,2295494,2304095
276,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,488599,492694,500444,504992,504992,504992,524716,535160,546176,556550
277,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,10585,10821,10888,10942,10942,10942,11019,11061,11113,11145
278,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,302569,303266,304002,304353,304656,304922,305047,305557,305959,306347


In [3]:
deaths_df


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/25/22,1/26/22,1/27/22,1/28/22,1/29/22,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,7397,7401,7403,7403,7405,7408,7414,7417,7418,7420
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,3312,3317,3323,3329,3329,3334,3346,3346,3362,3362
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,6516,6526,6539,6545,6555,6566,6579,6593,6604,6618
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,145,145,145,145,145,145,145,145,146,147
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,1891,1892,1893,1893,1894,1895,1895,1895,1895,1896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,NaN,Vietnam,14.058324,108.277199,0,0,0,0,0,0,...,37010,37165,37291,37432,37547,37668,37777,37777,37777,38063
276,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,5061,5072,5074,5076,5076,5076,5098,5104,5116,5128
277,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,1995,2003,2005,2006,2007,2011,2012,2013,2018,2020
278,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,3905,3906,3907,3910,3914,3915,3917,3920,3921,3925


In [4]:
#   full.to_csv('COVID-19-time-series-full-grouped.csv')
#   CHECK THIS OUT LATER *********************************************************************************************

In [5]:
recovered_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/25/22,1/26/22,1/27/22,1/28/22,1/29/22,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,NaN,Vietnam,14.058324,108.277199,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
261,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
262,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
#   save DFs to CSVs to do exploratory data analysis
confirmed_df.to_csv('conf_df.csv')
deaths_df.to_csv('deaths_df.csv')
recovered_df.to_csv('recvd_df.csv')


In [7]:
#   use melt() to unpivot DataFrames from current wide format into long format

dates = confirmed_df.columns[4:]
confirmed_df_long = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Confirmed'
)
deaths_df_long = deaths_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)
recovered_df_long = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Recovered'
)
confirmed_df_long

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
208315,NaN,Vietnam,14.058324,108.277199,2/3/22,2304095
208316,NaN,West Bank and Gaza,31.952200,35.233200,2/3/22,556550
208317,NaN,Yemen,15.552727,48.516388,2/3/22,11145
208318,NaN,Zambia,-13.133897,27.849332,2/3/22,306347


In [8]:
deaths_df_long

,Province/State,Country/Region,Lat,Long,Date,Deaths
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
208315,NaN,Vietnam,14.058324,108.277199,2/3/22,38063
208316,NaN,West Bank and Gaza,31.952200,35.233200,2/3/22,5128
208317,NaN,Yemen,15.552727,48.516388,2/3/22,2020
208318,NaN,Zambia,-13.133897,27.849332,2/3/22,3925


In [9]:
recovered_df_long

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0
...,...,...,...,...,...,...
197155,NaN,Vietnam,14.058324,108.277199,2/3/22,0
197156,NaN,West Bank and Gaza,31.952200,35.233200,2/3/22,0
197157,NaN,Yemen,15.552727,48.516388,2/3/22,0
197158,NaN,Zambia,-13.133897,27.849332,2/3/22,0


#### Remove recovered data for Canada due to mismatch issue
*    Canada recovered data is counted for the whole Country instead of by Province/State which is how Canada 
*        and the rest of the world count data for "Confirmed Cases" and "Deaths".

### we could apportion recovered data for the country in the same ratio as confirmed cases time permitting 


In [10]:
#   remove Canada recovered data
recovered_df_long = recovered_df_long[recovered_df_long['Country/Region']!='Canada']


####   merge the 3 data frames one after another

In [11]:

#   merge confirmed_df_long and deaths_df_long
full_table = confirmed_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

#    merge full_table and recovered_df_long
full_table = full_table.merge(
  right=recovered_df_long, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

full_table


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0.0
...,...,...,...,...,...,...,...,...
208315,NaN,Vietnam,14.058324,108.277199,2/3/22,2304095,38063,0.0
208316,NaN,West Bank and Gaza,31.952200,35.233200,2/3/22,556550,5128,0.0
208317,NaN,Yemen,15.552727,48.516388,2/3/22,11145,2020,0.0
208318,NaN,Zambia,-13.133897,27.849332,2/3/22,306347,3925,0.0


In [12]:
#   Check Canada data in "full_table" - "recovered" should be 0 and check of CSV file confirms it is 
full_table.to_csv('full_table.csv')

####   Data Cleansing
*    1. converting date from string to datetime
*    2. replacing missing value NaN with zeroes
*    3. coronavirus cases reported from 3 cruise ships should be treated differently

In [13]:
#   1 converting date from string to datetime
full_table['Date'] = pd.to_datetime(full_table['Date'])


In [14]:
full_table

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0
...,...,...,...,...,...,...,...,...
208315,NaN,Vietnam,14.058324,108.277199,2022-02-03,2304095,38063,0.0
208316,NaN,West Bank and Gaza,31.952200,35.233200,2022-02-03,556550,5128,0.0
208317,NaN,Yemen,15.552727,48.516388,2022-02-03,11145,2020,0.0
208318,NaN,Zambia,-13.133897,27.849332,2022-02-03,306347,3925,0.0


In [15]:
#   2 replacing missing values NaN
#   detect missing values NaN 
full_table.isna().sum()

Province/State    143592
Country/Region         0
Lat                 1488
Long                1488
Date                   0
Confirmed              0
Deaths                 0
Recovered          15624
dtype: int64

In [16]:
#   replace 'recovered' Nans with zero
full_table['Recovered'] = full_table['Recovered'].fillna(0)
full_table['Recovered']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
208315    0.0
208316    0.0
208317    0.0
208318    0.0
208319    0.0
Name: Recovered, Length: 208320, dtype: float64

####   3 coronavirus cases reported from 3 cruise ships should be treated differently
*    Cases reported from cruise ships: Grand Princess, Diamond Princess and MS Zaandam need to be extracted and 
*    treated differently due to Province/State and Country/Region mismatch over time

In [17]:
#   extract data for these ships 
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | full_table['Province/State'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('Diamond Princess') | full_table['Country/Region'].str.contains('MS Zaandam')
full_ship = full_table[ship_rows]

#   remove data for these ships 
full_table = full_table[~(ship_rows)]

In [18]:
#   check "full_table" to ensure its integrity
full_table.to_csv('full_table_clean.csv')

#### Calculate the number of Active Cases 
    *Active Cases = Confirmed Cases - Deaths - Recovered Cases 
    

In [19]:
#    calculate active cases = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']
full_table

C:\Users\mikem\AppData\Local\Temp/ipykernel_18200/2074956377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0.0,0.0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0.0,0.0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0.0,0.0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0.0,0.0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
208315,NaN,Vietnam,14.058324,108.277199,2022-02-03,2304095,38063,0.0,2266032.0
208316,NaN,West Bank and Gaza,31.952200,35.233200,2022-02-03,556550,5128,0.0,551422.0
208317,NaN,Yemen,15.552727,48.516388,2022-02-03,11145,2020,0.0,9125.0
208318,NaN,Zambia,-13.133897,27.849332,2022-02-03,306347,3925,0.0,302422.0


In [20]:
#   aggregate data into Country/Region and group by Date and Country/Region

full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
full_grouped

C:\Users\mikem\AppData\Local\Temp/ipykernel_18200/2747540925.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()


,Date,Country/Region,Confirmed,Deaths,Recovered,Active
0,2020-01-22,Afghanistan,0,0,0.0,0.0
1,2020-01-22,Albania,0,0,0.0,0.0
2,2020-01-22,Algeria,0,0,0.0,0.0
3,2020-01-22,Andorra,0,0,0.0,0.0
4,2020-01-22,Angola,0,0,0.0,0.0
...,...,...,...,...,...,...
144331,2022-02-03,Vietnam,2304095,38063,0.0,2266032.0
144332,2022-02-03,West Bank and Gaza,556550,5128,0.0,551422.0
144333,2022-02-03,Yemen,11145,2020,0.0,9125.0
144334,2022-02-03,Zambia,306347,3925,0.0,302422.0


In [21]:
#   calculate daily New cases, New deaths and New recovered by deducting the corresponding accumulative data on the previous day
#   new cases 
temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan

#   renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']

#   merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])

#   filling na with 0
full_grouped = full_grouped.fillna(0)

#   fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')

full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)


C:\Users\mikem\AppData\Local\Temp/ipykernel_18200/752199149.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']


####   Final output is data sorted by Date and Country/Region ascending where: -
*    Confirmed Cases, Deaths, Recovered and Active are cumulative data for the entire period, and,
*    New cases, New deaths and New Recovered are daily data

In [22]:
#    Final data frame
full_grouped

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
0,2020-01-22,Afghanistan,0,0,0.0,0.0,0,0,0
1,2020-01-22,Albania,0,0,0.0,0.0,0,0,0
2,2020-01-22,Algeria,0,0,0.0,0.0,0,0,0
3,2020-01-22,Andorra,0,0,0.0,0.0,0,0,0
4,2020-01-22,Angola,0,0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...
144331,2022-02-03,Vietnam,2304095,38063,0.0,2266032.0,8601,286,0
144332,2022-02-03,West Bank and Gaza,556550,5128,0.0,551422.0,10374,12,0
144333,2022-02-03,Yemen,11145,2020,0.0,9125.0,32,2,0
144334,2022-02-03,Zambia,306347,3925,0.0,302422.0,388,4,0


In [23]:
#   convert dataframe to a csv file for backup
full_grouped.to_csv('COVID-19-time-series-full-grouped.csv')


#### Check data for Australia against JHU Daily Replorts to confirm that it is correct


In [24]:
#   select Oz to check that data is correct
full_grouped_oz = full_grouped.loc[full_grouped["Country/Region"] == "Australia"]
full_grouped_oz

#   data is correct as at 03/02/2022

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered
8,2020-01-22,Australia,0,0,0.0,0.0,0,0,0
202,2020-01-23,Australia,0,0,0.0,0.0,0,0,0
396,2020-01-24,Australia,0,0,0.0,0.0,0,0,0
590,2020-01-25,Australia,0,0,0.0,0.0,0,0,0
784,2020-01-26,Australia,4,0,0.0,4.0,4,0,0
...,...,...,...,...,...,...,...,...,...
143374,2022-01-30,Australia,2607242,3760,0.0,2603482.0,36719,42,0
143568,2022-01-31,Australia,2580386,3835,0.0,2576551.0,0,75,0
143762,2022-02-01,Australia,2620532,3905,0.0,2616627.0,40146,70,0
143956,2022-02-02,Australia,2644760,3987,0.0,2640773.0,24228,82,0


#### VACCINATION DATASET SOURCED FROM OUR WORLD IN DATA AT THE FOLLOWING LINK
*    https://ourworldindata.org/covid-vaccinations


In [25]:
#   read the vaccination dataset - csv file into a dataframe
vacc_data_df = pd.read_csv('owid-covid-data.csv')
vacc_data_df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2/24/2020,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2/25/2020,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2/26/2020,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2/27/2020,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2/28/2020,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159320,ZWE,Africa,Zimbabwe,1/30/2022,229460.0,45.0,172.286,5337.0,4.0,6.143,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
159321,ZWE,Africa,Zimbabwe,1/31/2022,229666.0,206.0,160.714,5338.0,1.0,4.714,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
159322,ZWE,Africa,Zimbabwe,2/1/2022,229851.0,185.0,153.571,5350.0,12.0,4.857,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN
159323,ZWE,Africa,Zimbabwe,2/2/2022,230012.0,161.0,152.714,5352.0,2.0,4.429,...,1.6,30.7,36.791,1.7,61.49,0.571,NaN,NaN,NaN,NaN


#### WORLD POPULATION DATASET AS AT 31/12/2020 SOURCED FROM THE FOLLOWING LINK
*    https://www.worldometers.info/world-population/population-by-country/   

In [26]:


#   read the world population dataset - csv file into a dataframe
pop_data_df = pd.read_csv('world_population.csv')
pop_data_df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,...,102050.0,102565.0,103165.0,103776.0,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,...,532760424.0,547482863.0,562601578.0,578075373.0,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,...,30117411.0,31161378.0,32269592.0,33370804.0,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,...,360285439.0,370243017.0,380437896.0,390882979.0,401586651.0,412551299.0,423769930.0,435229381.0,446911598.0,458803476.0
4,Angola,AGO,"Population, total",SP.POP.TOTL,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,...,24220660.0,25107925.0,26015786.0,26941773.0,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,...,1791000.0,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0,1791003.0,1797085.0,1788878.0,1775378.0
262,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,5315351.0,5393034.0,5473671.0,5556767.0,5641598.0,5727745.0,...,23807586.0,24473176.0,25147112.0,25823488.0,26497881.0,27168210.0,27834811.0,28498683.0,29161922.0,29825968.0
263,South Africa,ZAF,"Population, total",SP.POP.TOTL,17099836.0,17524533.0,17965733.0,18423157.0,18896303.0,19384838.0,...,52003759.0,52832659.0,53687125.0,54544184.0,55386369.0,56207649.0,57009751.0,57792520.0,58558267.0,59308690.0
264,Zambia,ZMB,"Population, total",SP.POP.TOTL,3070780.0,3164330.0,3260645.0,3360099.0,3463211.0,3570466.0,...,14023199.0,14465148.0,14926551.0,15399793.0,15879370.0,16363449.0,16853608.0,17351714.0,17861034.0,18383956.0


#### THIS FILE WAS CREATED FROM THE VACCINATION DATASET SOURCED FROM OUR WORLD IN DATA AT THE FOLLOWING LINK
*    https://ourworldindata.org/covid-vaccinations
*    THE "PEOPLE_NOT_VACCINATED_PER_HUNDRED" WAS DERIVED FROM THE "PEOPLE_FULLY_VACCINATED_PER_HUNDRED"

In [30]:
#   load "percent-people-fully-vaccinated-covid"
percent_vaccinated_df = pd.read_csv('percent_people_fully_vaccinated.csv')
percent_vaccinated_df


FileNotFoundError: [Errno 2] No such file or directory: 'percent_people_fully_vaccinated.csv'

In [ ]:
#   data cleansing'Country/Region].replace(['United States'], 'US')
percent_vaccinated_df['Country/Region'] = percent_vaccinated_df['Country/Region'].replace(['United States'],'US')


In [ ]:
#   THIS DATASET WAS SOURCED FROM THE JHU MORTALITY SITE AT THE FOLLOWING LINK
#   https://coronavirus.jhu.edu/data/mortality

#   read the jhu_global_fatality csv file into a dataframe
jhu_global_fatality_df = pd.read_csv('jhu_global_fatality.csv')
jhu_global_fatality_df

In [ ]:
#   data cleansing'Country/Region].replace(['United States'], 'US')
jhu_global_fatality_df['Country/Region'] = jhu_global_fatality_df['Country/Region'].replace(['United States'],'US')


In [ ]:
#   create a data frame for the selected countries of interest
countries_df = full_grouped[full_grouped['Country/Region'].isin([ "Australia","US","United Kingdom","New Zealand","Italy","Israel", "Brazil", "South Africa"])]
full_grouped.dropna(axis=1, how="all")
countries_df 


In [ ]:
#   convert dataframe to a csv file for backup
countries_df.to_csv('Countries_Df.csv')


In [ ]:
#   merge "countries_df" with "jhu_global_fatality_df" 
merged_inner1 = pd.merge(left=countries_df, right=jhu_global_fatality_df, left_on='Country/Region', right_on='Country/Region')
merged_inner1


In [ ]:
#   convert dataframe to a csv file for backup
merged_inner1.to_csv('merged_inner1.csv')


In [ ]:
merged_inner2 = pd.merge(left=merged_inner1, right=percent_vaccinated_df, left_on='Country/Region', right_on='Country/Region')
merged_inner2


In [ ]:
#   change "merged_inner2" to analysis dataframe
analysis_df = merged_inner2
analysis_df


In [ ]:
#   replace Nans with 0
analysis_df.fillna(0)


In [ ]:
#   NOTE THE FOLLOWING CODE HAS BEEN DISABLED AS IT RESULTS IN A CSV FILE OF 160MB. DESPITE EXTENSIVE SUPPORT FROM ASKBCS AND
#   UNINSTALLING / REINSTALLING GIT THIS ISSUE HAS YET TO BE RESOLVED
#   TO BE FOLLOWED UP WITH TUTORS / LECTURERS FOR SOLUTION

#   convert dataframe to a csv file for backup
#analysis_df.to_csv('analysis_df.csv')

In [ ]:
#   ERIKA'S CODE & PLOTS REVIEWED AND AGREED BY ALL START HERE

In [ ]:
data = pd.read_csv("Analysis1.csv")
data


In [ ]:
df = data[data['Country_Region'].isin([ "Australia","US","United Kingdom","New Zealand","Italy","Israel", "Brazil", "South Africa"])]
df.dropna(axis=1, how="all")


In [ ]:
df = data[data['Country_Region'].isin([ "Australia","US","United Kingdom","New Zealand","Italy","Israel", "Brazil", "South Africa"])]
df.dropna(axis=1, how="all")


In [ ]:
plt.scatter(df['Latitude'],df['Confirmed'],alpha=0.5,marker="o",edgecolors='black')
plt.xlabel('Latitude')
plt.ylabel('Confirmed Cases')
plt.title('City Latitude vs. Confirmed Cases')
plt.grid()


In [ ]:
plt.scatter(df['Latitude'],df['Deaths'],alpha=0.8,marker="o",edgecolors='black')
plt.xlabel('Latitude')
plt.ylabel('Deaths')
plt.title('City Latitude vs. Deaths')
plt.grid()


In [ ]:
plt.scatter(df['Latitude'],df['Deaths'],alpha=0.8,marker="o",edgecolors='black')
plt.xlabel('Latitude')
plt.ylabel('Deaths')
plt.title('City Latitude vs. Deaths')
plt.grid()

x_values = df['Latitude']
y_values = df['Deaths']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x " + str(round(intercept,2))
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(0,100000),fontsize=15,color="red")


In [ ]:
data = pd.read_csv("vaccine_data_global.csv")
data


In [ ]:
df = data[data['Country_Region'].isin(["Australia","US","United Kingdom","New Zealand","Italy","Israel", "Brazil", "South Africa", "Vietnam"])]
df


In [ ]:
Country_Vaccination = pd.DataFrame(df.groupby('Country_Region')['People_fully_vaccinated'].agg('sum'))
Country_Vaccination.to_csv("Country Vaccination.csv")


In [ ]:
Bar_Pandas = Country_Vaccination.plot.bar(figsize=(10,5), rot=90)
plt.xlabel("Country", fontsize=12)
plt.ylabel("Number of Fully Vaccinated People", fontsize=12)
plt.title("Total Number of Fully Vaccinated People by Country", fontsize=15)
plt.savefig("Total Number of Fully Vaccinated People by Country.png")


In [ ]:
world_pop = pd.read_csv("world_population.csv")
us_df = world_pop[world_pop["Country Name"] == "United States"]
us_df


In [ ]:
world_pop = pd.read_csv("world_population.csv")
world_df = world_pop[world_pop["Country Name"].isin(["Australia","United States","United Kingdom","New Zealand","Italy","Israel", "Brazil", "South Africa", "Vietnam"])]
world_df


In [ ]:
World_Pop1 = pd.DataFrame(world_df.groupby('Country Name')['2020'].agg('sum'))
World_Pop = World_Pop1.rename(columns={'2020': 'Population'},index={'United States': 'US'})
World_Pop


In [ ]:
frames = [Country_Vaccination, World_Pop]
df = pd.concat(frames, axis=1)
df['Percentage_Vacc'] = (df['People_fully_vaccinated'] / df['Population'])
df['Percentage_Vacc'] = df['Percentage_Vacc'].apply(lambda x: x*100)
df.style.format({'Percentage_Vacc': "{:.2%}"})
df.to_csv('Country Vaccination.csv', index=False)


In [ ]:
xaxis = ["Australia", "Brazil", "Israel", "Italy", "New Zealand", "South Africa", "US", "United Kingdom", "Vietnam"]
yaxis = df["Percentage_Vacc"]
plt.xticks(rotation=90)
ax = plt.bar(xaxis, yaxis)
plt.xlabel("Country Name")
plt.ylabel("Percentage Fully Vaccinated (%)")
plt.title("Percentage of Population Fully Vaccinated by Country")
plt.savefig("Percentage of Population Fully Vaccinated by Country.png")

#label = df['Percentage_Vacc']
#plt.annotate(
    #list(label),                      # Use `label` as label
    #(xaxis, yaxis),         # Place label at end of the bar
    #xytext=(5, 0),          # Horizontally shift label by `space`
    #textcoords="offset points", # Interpret `xytext` as offset in points
    #va='center',                # Vertically center label
    #ha='left') 


In [ ]:
labels = ["Fully Vaccinated", "Unvaccinated"]
sizes = [19375276, (25687041-19375276)]
colors = ["lightskyblue", "red"]
explode = (0.1, 0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True)
plt.title("Proportion of Population Fully Vaccinated vs Unvaccinated in Australia")
plt.savefig("Proportion of Population Fully Vaccinated vs Unvaccinated in Australia.png")


In [ ]:
labels = ["Fully Vaccinated", "Unvaccinated"]
sizes = [140151965, (212559409-140151965)]
colors = ["blue", "orange"]
explode = (0.1, 0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True)
plt.title("Proportion of Population Fully Vaccinated vs Unvaccinated in Brazil")
plt.savefig("Proportion of Population Fully Vaccinated vs Unvaccinated in Brazil.png")


In [ ]:
labels = ["Fully Vaccinated", "Unvaccinated"]
sizes = [5798867, (9216900-5798867)]
colors = ["green", "yellow"]
explode = (0.1, 0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True)
plt.title("Proportion of Population Fully Vaccinated vs Unvaccinated in Israel")
plt.savefig("Proportion of Population Fully Vaccinated vs Unvaccinated in Israel.png")


In [ ]:
labels = ["Fully Vaccinated", "Unvaccinated"]
sizes = [44343143, (59554023-44343143)]
colors = ["pink", "orange"]
explode = (0.1, 0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True)
plt.title("Proportion of Population Fully Vaccinated vs Unvaccinated in Italy")
plt.savefig("Proportion of Population Fully Vaccinated vs Unvaccinated in Italy.png")


In [ ]:
labels = ["Fully Vaccinated", "Unvaccinated"]
sizes = [3766864, (5084300-3766864)]
colors = ["purple", "lightblue"]
explode = (0.1, 0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True)
plt.title("Proportion of Population Fully Vaccinated vs Unvaccinated in New Zealand")
plt.savefig("Proportion of Population Fully Vaccinated vs Unvaccinated in New Zealand.png")


In [ ]:
labels = ["Fully Vaccinated", "Unvaccinated"]
sizes = [15410168, (59308690-15410168)]
colors = ["lightblue", "green"]
explode = (0.1, 0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True)
plt.title("Proportion of Population Fully Vaccinated vs Unvaccinated in South Africa")
plt.savefig("Proportion of Population Fully Vaccinated vs Unvaccinated in South Africa.png")


In [ ]:
labels = ["Fully Vaccinated", "Unvaccinated"]
sizes = [202845886, (329484123-202845886)]
colors = ["yellow", "red"]
explode = (0.1, 0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True)
plt.title("Proportion of Population Fully Vaccinated vs Unvaccinated in the US")
plt.savefig("Proportion of Population Fully Vaccinated vs Unvaccinated in US.png")


In [ ]:
labels = ["Fully Vaccinated", "Unvaccinated"]
sizes = [46804609, (67215293-46804609)]
colors = ["orange", "lightblue"]
explode = (0.1, 0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True)
plt.title("Proportion of Population Fully Vaccinated vs Unvaccinated in UK")
plt.savefig("Proportion of Population Fully Vaccinated vs Unvaccinated in UK.png")


In [ ]:
labels = ["Fully Vaccinated", "Unvaccinated"]
sizes = [53224715, (97338583-53224715)]
colors = ["pink", "red"]
explode = (0.1, 0)
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True)
plt.title("Proportion of Population Fully Vaccinated vs Unvaccinated in Vietnam")
plt.savefig("Proportion of Population Fully Vaccinated vs Unvaccinated in Vietnam.png")


In [ ]:
death_df = pd.read_csv("Country Death.csv")
death_df.set_index("Country_Region", inplace = True)
death_df


In [ ]:
#df.to_csv("Percentage Vaccinated.csv")


In [ ]:
frames = [death_df, df]
df1 = pd.concat(frames, axis=1)
df_clean = df1.loc[:,~df1.columns.duplicated()]
df_clean


In [ ]:
df_clean['Percentage_Death'] = ((df_clean['Deaths'] / df_clean['Population'])*100 )
#df_clean['Percentage_Death'] = df_clean['Percentage_Death'].apply(lambda x: x*100)
#df_clean.style.format({'Percentage_Death': "{:.2%}"})
df_clean


In [ ]:
x_axis = df_clean["Percentage_Death"]
y_axis = df_clean["Percentage_Vacc"]
plt.xlabel("Percentage of Population Death")
plt.ylabel("Percentage of Population Vaccinated")
plt.scatter(x_axis, y_axis, marker="o")

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
regress_values = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x " + str(round(intercept,2))
plt.plot(x_axis,regress_values,"r-")
plt.annotate(line_eq,(0,30),fontsize=15,color="red")
print(f"The r-squared value is: {rvalue**2}")
print(f"The correlation between Percentage of Population Death and Vaccinated is: {rvalue}")



In [ ]:
#   JOHN'S & MIKE'S CODE REVIEWED AND AGREED BY ALL STARTS HERE

In [ ]:
#   plot mortality for selected countries of interest using pyplot
mortality = analysis_df.groupby(["Country/Region"]).max()["Deaths"]
plt.bar(mortality.index.values,mortality.values)
plt.xlabel("Country")
plt.xticks(rotation = 90)
plt.ylabel("Mortality")
plt.title("Mortality per Country", fontsize=15)
fig1 = plt.gcf()
plt.show()
#   save as a png
fig1.savefig("Mortality.png")



In [ ]:
#   plot "deaths per 100,000 people" for selected countries of interest using pyplot
Deaths_per_100k = analysis_df.groupby(["Country/Region"]).max()["Deaths_per_100k"]
plt.bar(Deaths_per_100k.index.values,Deaths_per_100k.values)
plt.xlabel("Country")
plt.xticks(rotation = 90)
plt.ylabel("Deaths per 100,000 People")
plt.title("Number of Deaths per 100,000 People per Country", fontsize=15)
fig1 = plt.gcf()
plt.show()
#   save as a png
fig1.savefig("Deaths_per_100k_People.png")


In [ ]:
#   plot "case fatality ratio" for selected countries of interest using pyplot
Case_Fatality = analysis_df.groupby(["Country/Region"]).max()["Case_Fatality"]
plt.bar(Case_Fatality.index.values,Case_Fatality.values)
plt.xlabel("Country")
plt.xticks(rotation = 90)
plt.ylabel("Case Fatality Ratio")
plt.title("Case Fatality Ratio per Country", fontsize=15)
fig1 = plt.gcf()
plt.show()
#   save as a png
fig1.savefig("Case_Fatality_Ratio.png")



In [ ]:
def linearRegression(x_values,y_values,regEQplotX,regEQplotY,y_label):
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    print(f'The r-square is {rvalue}')
    print(line_eq)
    plt.annotate(line_eq,(regEQplotX,regEQplotY),fontsize=15,color="red")
    plt.xlabel(f'{x_label}')
    plt.ylabel(f'{y_label}')
    plt.grid()

In [ ]:
#   Calculate the correlation coefficient
#   for people_fully_vaccinated_per_hundred and case-fatality_ratio for selected countries
corr=round(st.pearsonr(analysis_df['Case_Fatality'],analysis_df['people_fully_vaccinated_per_hundred'])[0],2)
print(f"The correlation between people fully vaccinated per hundred and deaths is {corr}")
model = st.linregress(analysis_df['Case_Fatality'],analysis_df['people_fully_vaccinated_per_hundred'])

y_values = analysis_df['Case_Fatality']*model[0]+model[1]
plt.scatter(analysis_df['Case_Fatality'],analysis_df['people_fully_vaccinated_per_hundred'])
plt.plot(analysis_df['Case_Fatality'],y_values,color="red")
plt.xlabel('people_fully_vaccinated_per_hundred')
plt.ylabel('Case/Fatality Ratio')
plt.show()


#save as a png
plt.savefig("Percentage of People Fully Vaccinated versus Case Fatality Ratio.png")

#   the pearson's correlation coefficient 0f -0.15 indicates a negligible level of correlation
#   need  to check the t value and the p-value to determine significance

In [ ]:
#   Calculate the correlation coefficient
#   for people_fully_vaccinated_per_hundred and deaths_per100k for selected countries
corr=round(st.pearsonr(analysis_df['Deaths_per_100k'],analysis_df['people_fully_vaccinated_per_hundred'])[0],2)
print(f"The correlation between people fully vaccinated per hundred and deaths is {corr}")
model = st.linregress(analysis_df['Deaths_per_100k'],analysis_df['people_fully_vaccinated_per_hundred'])

y_values = analysis_df['Deaths_per_100k']*model[0]+model[1]
plt.scatter(analysis_df['Deaths_per_100k'],analysis_df['people_fully_vaccinated_per_hundred'])
plt.plot(analysis_df['Deaths_per_100k'],y_values,color="red")
plt.xlabel('people_fully_vaccinated_per_hundred')
plt.ylabel('Deaths/100k')
plt.show()

#save as a png
plt.savefig("Percentage of People Fully Vaccinated versus Deaths per 100k.png")

#   the pearson's correlation coefficient 0f 0.11 indicates a negligible level of correlation
#   need  to check the p-value to determine significance